In [ ]:
#back reaction

import matplotlib.pyplot as plt 
from matplotlib.ticker import FuncFormatter
import numpy as np

#%% 

from astropy import constants as ast

m_sol = ast.M_sun.cgs.value   # masa del sol en gramos
G = ast.G.cgs.value           #constante de gravitacion universal en cgs
c = ast.c.cgs.value           #velocidad de la luz en cgs
#%%


from scipy import constants as sci
pi = sci.pi

#%%
eta_0 = 2.96e12 #son aproximadamente los 47k anios donde domina la rad
eta_b = 21969.3   #escala del bounce
a_b = 7.41155e-9 #factor de normalizacion del factor de escala
xini = eta_0/eta_b  #cte inicial en funcion de x

#definimos cte de normalizacion 
rho_rad_ini = (3*c**2/(8*pi*G))*(1/(a_b*eta_b)**2)*xini**2*(1+xini**2)**(-3)  #dens de rad inicial [erg/cm^3]
beta =1e-10
rho_bh_ini = beta*rho_rad_ini   

alfa = 2.36
M_min = 1e16
M_max = m_sol*1e6

cteA = (rho_bh_ini*(2-alfa))/(c**2*(M_max**(2-alfa)-M_min**(2-alfa)))


In [ ]:
import pandas as pd

from fractions import Fraction

In [ ]:
num_eta = 100000

eta_pos = np.logspace(-20, 13, num_eta)
eta_neg = -eta_pos[::-1] 
eta_int = np.concatenate((eta_neg,[0], eta_pos))
x_int = eta_int/eta_b

In [ ]:
rho_bh = np.zeros_like(eta_int) 

Nm = 100
Masas = np.logspace(np.log10(M_min), np.log10(M_max),Nm)

hlog = (M_max/M_min)**(1/(Nm-1))

i = 0
for x in x_int:
    a = a_b*(1+x**2)**(1/2) #factor de escala
    for M in Masas:
        M_i = M*a
        N_i = cteA*M**(-alfa)*a**(-3)   #densidad inicial de bhs en el bloque i

        delta = M*(hlog-1)

        rho_bh[i] = rho_bh[i]+ N_i*M_i*c**2*delta
        
    i = i+1

rho_bh_norm = rho_bh/rho_bh_ini

In [ ]:
ctes1 = ((8*pi*G)/(3*c**2)**(1/2))

ctes2 = -(4*pi*G)/3

In [ ]:
#defino varicion de la densidad de energia unicamente debido a la interaccion entre los fluidos

delta_rho = np.zeros_like(eta_int) 
f_escala =  np.zeros_like(eta_int) 
da = np.zeros_like(eta_int)
dda = np.zeros_like(eta_int)
rho_cf = np.zeros_like(eta_int)
P_cf = np.zeros_like(eta_int)

i = 0  

for x in x_int: 
    delta_x = x_int[i+1]-x_int[i]

    da[i] = f_escala[i]**2*ctes1*(rho_bh[i]+rho_cf[i])**(1/2)

    omega_cf = 1/3-2/(3*x**2)
    P_cf[i] = c**2*omega_cf*rho_cf[i]
    dda[i] = ctes2*(rho_bh[i]+rho_cf[i]+3*P_cf[i]/c**2)*f_escala[i]**2 + da[i]**2/f_escala[i]**2

    f_escala[i+1] = f_escala[i] + da[i]*delta_x + (1/2)*dda[i]*(delta_x)**2
    
    delta_rho[i+1] = rho_bh[i+1] - rho_bh[i]*(f_escala)